In [ ]:
!pip install llama-index-core llama-index-llms-groq llama-index-embeddings-huggingface gradio pandas


In [ ]:
import os
import pandas as pd
import gradio as gr

from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


In [ ]:
from google.colab import files


uploaded = files.upload()


In [ ]:
import pandas as pd

df = pd.read_csv("menu.csv")


print("Shape:", df.shape)
print(df.columns.tolist())
print(df.head())


In [ ]:
import os
from llama_index.llms.groq import Groq
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


os.getenv("GROQ_API_KEY")



llm = Groq(model="llama-3.1-8b-instant", temperature=0.2, max_tokens=512)
Settings.llm = llm

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.embed_model = embed_model


In [ ]:
from llama_index.core import Document

documents = []

for _, row in df.iterrows():
    text = f"""
    Item ID: {row['item_id']}
    Name: {row['name']}
    Category: {row['category']}
    Price: {row['price_lkr']}
    Ingredients: {row['ingredients']}
    Dietary: {row['dietary']}
    Spice Level: {row['spice_level']}
    Description: {row['description']}
    """
    documents.append(Document(text=text))


In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()


print(query_engine.query("What are your vegetarian options under 1500 LKR?"))
print(query_engine.query("What’s your cheapest dessert?"))


In [ ]:
import gradio as gr

def chatbot(message, history):

    response = query_engine.query(message)
    return str(response)


demo = gr.ChatInterface(
    fn=chatbot,
    title="🍴 Restaurant Menu Chatbot",
    description="Ask me about our menu: prices, ingredients, dietary tags, spice levels, or get recommendations!"
)

demo.launch(share=True)


# Restaurant Menu Chatbot – Reflection

- Parameters used  
  - LLM: `llama-3.1-8b-instant` (Groq).  
  - `temperature = 0.2` → keeps answers factual and consistent.  
  - `max_tokens = 512` → allows detailed explanations when needed.  

- Indexing approach  
  - Each menu item (row in `menu.csv`) was converted into a `Document`.  
  - Used **HuggingFace embeddings (`all-MiniLM-L6-v2`)** for semantic search.  
  - Built a `VectorStoreIndex` to enable retrieval-augmented generation (RAG).  

*EXAMPLE SCREENSHOTS

In [ ]:
from google.colab import files
from IPython.display import Image


uploaded = files.upload()
display(Image("Chatbot1.png"))


In [ ]:
from google.colab import files
from IPython.display import Image


uploaded = files.upload()
display(Image("Chatbot2.png"))


In [ ]:
from google.colab import files
from IPython.display import Image


uploaded = files.upload()
display(Image("Chatbot3.png"))


In [ ]:
from google.colab import files
from IPython.display import Image


uploaded = files.upload()
display(Image("Chatbot4.png"))


- Limitations  
  - Dataset is small (40 items), so recommendations are simple.  
  - Responses depend on how clearly data is structured in the CSV.  
  - No memory → chatbot doesn’t remember past conversations.  
  - Menu updates require re-uploading/re-indexing the CSV.

**Summary**

I have built a simple chatbot using Llama Index and Groq LLM. The given restuarant menu dataset was loaded then converted into documents. VectorStoreIndex was connected to Groq Llama-3.1-8b model to answer queries. Gradio was used for the user interaction.



Challenges faced

I struggled to find a supported Groq model.
I used HuggingFace embeddings since default OpenAI embeddings caused errors.


Outcomes

 The chatbot can answer menu-related questions such as dietary preferences, price filtering, spice levels, and combo suggestions.  
  - Achieved good response accuracy with low temperature for factual consistency.  
  - Demonstrated the use of RAG for a small restaurant dataset.  
  - The system is lightweight, extendable, and deployable in real use cases.  